In [10]:
import numpy as np
import cv2

In [11]:
# reorders all the points accordingly
def rectify(corner_points):
    corner_points = corner_points.reshape((4, 2))
    new_points = np.zeros((4, 2), dtype=np.float32)
    temp1 = corner_points.sum(1)
    new_points[0] = corner_points[np.argmin(temp1)]
    new_points[2] = corner_points[np.argmax(temp1)]
    temp2 = np.diff(corner_points, axis=1)
    new_points[1] = corner_points[np.argmin(temp2)]
    new_points[3] = corner_points[np.argmax(temp2)]
    return new_points

In [12]:
# returns the corner points of the document, from all the possible contours
def get_rectangle(contours):
    contours = sorted(contours, key=cv2.contourArea, reverse=True)
    for contour in contours:
        p = cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, 0.02 * p, True)

        if len(approx) == 4:
            return approx

In [13]:
# shows all required images by accepting an image array
def show_images(images):
    names = (["Initial image", "Scanned output, coloured", "Edged image",
              "Scanned threshold, binary"])
    for i in range(len(images)):
        cv2.imshow(names[i], images[i])

    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [14]:
# finds the binary image of the document through adaptive threshold
def get_binary(scanned_image):
    scanned_image_grey = cv2.cvtColor(scanned_image, cv2.COLOR_BGR2GRAY)
    # scanned_image_grey = cv2.medianBlur(scanned_image_grey, 5)
    scanned_image_thresh_binary = cv2.adaptiveThreshold(scanned_image_grey, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                                        cv2.THRESH_BINARY, 25, 5)
    return scanned_image_thresh_binary

In [15]:
# the main function, that calls all other functions
def get_scanned_image(image, width, height):
    # part 1: resizing our image
    image = cv2.resize(image, (width, height))

    # part 2: converting our image to get edges
    grey_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred_image = cv2.GaussianBlur(grey_image, (5, 5), 0)
    edged_image = cv2.Canny(blurred_image, 0, 50)

    # part 3: finding contours in our edged image
    contours, _ = cv2.findContours(edged_image, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

    # finding the corner points and reordering them
    rectangle = get_rectangle(contours)
    approx_rectangle = np.float32(rectify(rectangle))

    # part 5: finding the bird's eye view of the document
    dimensions = np.float32([[0, 0], [width, 0], [width, height], [0, height]])
    matrix = cv2.getPerspectiveTransform(approx_rectangle, dimensions)
    scanned_image = cv2.warpPerspective(image, matrix, (width, height))

    # part 6: drawing the rectangle detected in our image
    cv2.drawContours(image, [rectangle], -1, (0, 255, 0), 3)

    # getting the binary image of the final scan
    scanned_image_thresh_binary = get_binary(scanned_image)
    images = ([image, scanned_image, edged_image, scanned_image_thresh_binary])
    return images

In [16]:
img = cv2.imread('doc4.jpeg')
imgs = get_scanned_image(img, 600, 800)
show_images(imgs)